# Set up

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from ortools.linear_solver import pywraplp
import numpy as np
import os

<IPython.core.display.Javascript object>

In [5]:
def main():
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP','SCIP')

    infinity = solver.infinity()
    # x and y are integer non-negative variables.
    x = solver.IntVar(0, infinity, 'x')
    y = solver.IntVar(0, infinity, 'y')

    print('Number of variables =', solver.NumVariables())

    # x + 7 * y <= 17.5.
    solver.Add(x + 7 * y <= 17.5)

    # x <= 3.5.
    solver.Add(x <= 3.5)

    print('Number of constraints =', solver.NumConstraints())

    # Maximize x + 10 * y.
    solver.Maximize(x + 10 * y)

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        print('x =', x.solution_value())
        print('y =', y.solution_value())
    else:
        print('The problem does not have an optimal solution.')

    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())




<IPython.core.display.Javascript object>

In [4]:
main()

Number of variables = 2
Number of constraints = 2
Solution:
Objective value = 23.0
x = 3.0
y = 2.0

Advanced usage:
Problem solved in 11.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes


<IPython.core.display.Javascript object>

# Knapsack

In [24]:
path = '../2 knapsack/data'
files = os.listdir(path)

<IPython.core.display.Javascript object>

In [25]:
for i, f in enumerate(files):
    print(i, f)

0 ks_10000_0
1 ks_1000_0
2 ks_100_0
3 ks_100_1
4 ks_100_2
5 ks_106_0
6 ks_19_0
7 ks_200_0
8 ks_200_1
9 ks_300_0
10 ks_30_0
11 ks_400_0
12 ks_40_0
13 ks_45_0
14 ks_4_0
15 ks_500_0
16 ks_50_0
17 ks_50_1
18 ks_60_0
19 ks_82_0
20 ks_lecture_dp_1
21 ks_lecture_dp_2


<IPython.core.display.Javascript object>

In [26]:
def parse_data(data):
    lines = data.split("\n")
    n, cap = lines[0].split()
    data = [line.split() for line in lines if len(line) > 0]
    n, cap = data.pop(0)
    return np.int(n), np.float(cap), np.array(data, dtype=float)

<IPython.core.display.Javascript object>

In [40]:
file = files[0]
with open(os.path.join(path, file)) as fp:
    data = fp.read()

n, K, data = parse_data(data)
values = data[:, 0]
weights = data[:, 1]

<IPython.core.display.Javascript object>

In [38]:
def ks_mip(weights, values, capacity):
    assert len(weights) == len(values), "Weights and Values must be of the same size."

    # Create solver
    solver = pywraplp.Solver.CreateSolver("SCIP", "SCIP")

    # Define Variables
    n = len(weights)
    choices = [solver.BoolVar(f"x_{i}") for i in range(n)]

    # Constraints
    total_weight = sum([weights[i] * choices[i] for i in range(n)])
    solver.Add(total_weight <= capacity)

    # Objective
    total_value = sum([values[i] * choices[i] for i in range(n)])
    solver.Maximize(total_value)

    # Solve
    status = solver.Solve()

    # Solution
    if status == pywraplp.Solver.OPTIMAL:
        solution = [choices[i].solution_value() for i in range(n)]
        total_weight = sum([weights[i] * solution[i] for i in range(n)])
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value()}")
        print(f"Weight = {total_weight}/{capacity}")
        print(solution)
    #         print('x =', x.solution_value())
    #         print('y =', y.solution_value())
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())

<IPython.core.display.Javascript object>

In [ ]:
ks_mip(weights, values, capacity=K)

# Graph Coloring

In [6]:
path = "../3 coloring/data"
files = os.listdir(path)
[f"{i}. {f}" for i, f in enumerate(files)]

['0. gc_1000_1',
 '1. gc_1000_3',
 '2. gc_1000_5',
 '3. gc_1000_7',
 '4. gc_1000_9',
 '5. gc_100_1',
 '6. gc_100_3',
 '7. gc_100_5',
 '8. gc_100_7',
 '9. gc_100_9',
 '10. gc_20_1',
 '11. gc_20_3',
 '12. gc_20_5',
 '13. gc_20_7',
 '14. gc_20_9',
 '15. gc_250_1',
 '16. gc_250_3',
 '17. gc_250_5',
 '18. gc_250_7',
 '19. gc_250_9',
 '20. gc_4_1',
 '21. gc_500_1',
 '22. gc_500_3',
 '23. gc_500_5',
 '24. gc_500_7',
 '25. gc_500_9',
 '26. gc_50_1',
 '27. gc_50_3',
 '28. gc_50_5',
 '29. gc_50_7',
 '30. gc_50_9',
 '31. gc_70_1',
 '32. gc_70_3',
 '33. gc_70_5',
 '34. gc_70_7',
 '35. gc_70_9']

<IPython.core.display.Javascript object>

In [31]:
idx = 26
print(files[idx])
with open(os.path.join(path, files[idx]), "r") as fp:
    input_data = fp.read()
print(input_data[:15])

gc_50_1
50 133
0 5
0 15


<IPython.core.display.Javascript object>

In [32]:
lines = input_data.split("\n")

first_line = lines[0].split()
node_count = int(first_line[0])
edge_count = int(first_line[1])

edges = []
for i in range(1, edge_count + 1):
    line = lines[i]
    parts = line.split()
    edges.append((int(parts[0]), int(parts[1])))

print(node_count, edge_count)

50 133


<IPython.core.display.Javascript object>

In [33]:
edge_array = np.array(edges)
orders = [np.sum(edge_array == i) for i in range(node_count)]
highest_order = int(np.argmax(orders))
print(highest_order, orders[highest_order])

47 12


<IPython.core.display.Javascript object>

In [34]:
import networkx as nx
import matplotlib.pyplot as plt


def create_graph(n_points, edges):
    graph = nx.Graph()
    graph.add_nodes_from(range(n_points))
    graph.add_edges_from(edges)
    return graph


def get_max_clique(n_points, edges):
    graph = create_graph(n_points, edges)
    cliques = nx.find_cliques(graph)
    return max([len(c) for c in cliques])


def get_cliques(n_points, edges):
    graph = create_graph(n_points, edges)
    cliques = nx.find_cliques(graph)
    return [c for c in cliques if len(c) > 2]


def gc_mip(n_points, edges, max_color):
    n_colors = max_color
    min_colors = get_max_clique(n_points, edges)
    # Create solver
    solver = pywraplp.Solver.CreateSolver("SCIP", "SCIP")

    # Define Variables
    colors = [
        [solver.BoolVar(f"x_{i}_{c}") for c in range(n_colors)] for i in range(n_points)
    ]

    # Constraints
    # Connected edges
    for i, j in edges:
        for c in range(n_colors):
            solver.Add(colors[i][c] + colors[j][c] <= 1)

    # Only one color
    for i in range(n_points):
        solver.Add(sum(colors[i]) <= 1)
        solver.Add(sum(colors[i]) >= 1)

        #     total_weight = sum([weights[i] * choices[i] for i in range(n)])
        #     solver.Add(total_weight <= capacity)

        # Break Symmetry
    #         for c in range(n_colors - 1):
    #             sum1 = sum([colors[i][c] for i in range(n_points)])
    #             sum2 = sum([colors[i][c + 1] for i in range(n_points)])
    #             solver.Add(sum1 >= sum2)

    #             # Min colors
    #             if c < min_colors:
    #                 solver.Add(sum1 >= 1)

    # Use cliques as constraint
    cliques = get_cliques(n_points, edges)
    for clq in cliques:
        for c in range(n_colors):
            solver.Add(sum([colors[i][c] for i in clq]) <= 1)
    # Objective
    obj = 0
    for i in range(n_points):
        obj += sum([colors[i][c] * c for c in range(n_colors)])
    solver.Minimize(obj)

    # Solve
    status = solver.Solve()

    # Solution
    if status == pywraplp.Solver.OPTIMAL:
        #         solution = [choices[i].solution_value() for i in range(n)]
        solution = [
            [colors[i][c].solution_value() for c in range(n_colors)]
            for i in range(n_points)
        ]
        for i in range(n_points):
            print(solution[i])
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value()}")

    #         print(solution)
    #         print('x =', x.solution_value())
    #         print('y =', y.solution_value())
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())

<IPython.core.display.Javascript object>

In [35]:
sol = greedy_solver(edges, node_count)
max_color = len(set(sol))
max_color

5

<IPython.core.display.Javascript object>

In [ ]:
gc_mip(node_count, edges, max_color)

In [23]:
from graph_coloring import greedy_solver, cp_solver_test

<IPython.core.display.Javascript object>

In [22]:
greedy_solver(edges, node_count)

array([1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1])

<IPython.core.display.Javascript object>

In [24]:
cp_solver_test(edges,node_count)

	>>> Solver status:  OPTIMAL


[0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 3, 0, 1, 1, 0, 2, 2, 0, 0]

<IPython.core.display.Javascript object>

# Travelling Salesman

In [7]:
from collections import namedtuple

<IPython.core.display.Javascript object>

In [8]:
Point = namedtuple("Point", ["x", "y"])

<IPython.core.display.Javascript object>

In [3]:
path = "../4 tsp/data"
files = os.listdir(path)
[f"{i}. {f}" for i, f in enumerate(files)]

['0. tsp_1000_1',
 '1. tsp_100_1',
 '2. tsp_100_2',
 '3. tsp_100_3',
 '4. tsp_100_4',
 '5. tsp_100_5',
 '6. tsp_100_6',
 '7. tsp_101_1',
 '8. tsp_105_1',
 '9. tsp_1060_1',
 '10. tsp_107_1',
 '11. tsp_1084_1',
 '12. tsp_1173_1',
 '13. tsp_11849_1',
 '14. tsp_124_1',
 '15. tsp_127_1',
 '16. tsp_1291_1',
 '17. tsp_1304_1',
 '18. tsp_1323_1',
 '19. tsp_136_1',
 '20. tsp_1379_1',
 '21. tsp_1400_1',
 '22. tsp_14051_1',
 '23. tsp_1432_1',
 '24. tsp_144_1',
 '25. tsp_150_1',
 '26. tsp_150_2',
 '27. tsp_152_1',
 '28. tsp_1577_1',
 '29. tsp_159_1',
 '30. tsp_1655_1',
 '31. tsp_1748_1',
 '32. tsp_1817_1',
 '33. tsp_18512_1',
 '34. tsp_1889_1',
 '35. tsp_195_1',
 '36. tsp_198_1',
 '37. tsp_200_1',
 '38. tsp_200_2',
 '39. tsp_2103_1',
 '40. tsp_2152_1',
 '41. tsp_225_1',
 '42. tsp_226_1',
 '43. tsp_2319_1',
 '44. tsp_2392_1',
 '45. tsp_262_1',
 '46. tsp_264_1',
 '47. tsp_299_1',
 '48. tsp_3038_1',
 '49. tsp_318_1',
 '50. tsp_318_2',
 '51. tsp_33810_1',
 '52. tsp_3795_1',
 '53. tsp_400_1',
 '54. tsp

<IPython.core.display.Javascript object>

In [5]:
idx = 67
print(files[idx])
with open(os.path.join(path, files[idx]), "r") as fp:
    input_data = fp.read()
print(input_data[:35])

tsp_70_1
70
67 99
48 83
75 81
8 19
20 18
54 


<IPython.core.display.Javascript object>

In [11]:
lines = input_data.split("\n")

nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount + 1):
    line = lines[i]
    parts = line.split()
    points.append(Point(float(parts[0]), float(parts[1])))


<IPython.core.display.Javascript object>

[Point(x=67.0, y=99.0),
 Point(x=48.0, y=83.0),
 Point(x=75.0, y=81.0),
 Point(x=8.0, y=19.0),
 Point(x=20.0, y=18.0),
 Point(x=54.0, y=38.0),
 Point(x=63.0, y=36.0),
 Point(x=44.0, y=33.0),
 Point(x=52.0, y=18.0),
 Point(x=12.0, y=13.0),
 Point(x=25.0, y=5.0),
 Point(x=58.0, y=85.0),
 Point(x=5.0, y=67.0),
 Point(x=90.0, y=9.0),
 Point(x=41.0, y=76.0),
 Point(x=25.0, y=76.0),
 Point(x=37.0, y=64.0),
 Point(x=56.0, y=63.0),
 Point(x=10.0, y=55.0),
 Point(x=98.0, y=7.0),
 Point(x=16.0, y=74.0),
 Point(x=89.0, y=60.0),
 Point(x=48.0, y=82.0),
 Point(x=81.0, y=76.0),
 Point(x=29.0, y=60.0),
 Point(x=17.0, y=22.0),
 Point(x=5.0, y=45.0),
 Point(x=79.0, y=70.0),
 Point(x=9.0, y=100.0),
 Point(x=17.0, y=82.0),
 Point(x=74.0, y=67.0),
 Point(x=10.0, y=68.0),
 Point(x=48.0, y=19.0),
 Point(x=83.0, y=86.0),
 Point(x=84.0, y=94.0),
 Point(x=64.0, y=96.0),
 Point(x=80.0, y=39.0),
 Point(x=69.0, y=23.0),
 Point(x=72.0, y=42.0),
 Point(x=48.0, y=67.0),
 Point(x=58.0, y=43.0),
 Point(x=81.0, y=34.0)

<IPython.core.display.Javascript object>